- Env: py-39-2: 不能够运行  --jit --ipex，因此，不使用之
    - innersource-ipex : 2.1.0+git9608313
    - pip 4.28.1 
```
 ---------- Summary: ----------
Inference latency: 14.924 sec.
====================
prim_kind  impl      ncalls  time(ms)   overall%  agg_ncalls(avg)  agg_time(ms)  agg_overall%
matmul     gemm:jit  108000  216442.62  100.00    108000.00        216442.62     100.00
```

In [5]:
!ONEDNN_VERBOSE=1 python run_generation.py --device cpu --input-tokens=32 --max-new-tokens 32 \
    --num-iter=15 --num-warmup=5 --batch-size=1 -m EleutherAI/gpt-j-6b \
    --dtype bfloat16 > /tmp/gptj-2.log 2>/dev/null

In [8]:
!tail -n 5 /tmp/gptj-2.log
!echo ====================
!/home/taosy/repo/github/oneDNN/scripts/verbose_converter/verbose_converter.py -i /tmp/gptj-2.log -g breakdown -k prim_kind  impl | column -t -s,

['Once upon a time, there existed a little girl, who liked to have adventures. She wanted to go to places and meet new people, and have fun. One day, she decided to go on an adventure. She packed her bags, and set off on her journey.\n\nThe little girl walked and walked,'] [32]
Iteration: 14, Time: 13.725374 sec

 ---------- Summary: ----------
Inference latency: 14.924 sec.


prim_kind  impl      ncalls  time(ms)   overall%  agg_ncalls(avg)  agg_time(ms)  agg_overall%
matmul     gemm:jit  108000  216442.62  100.00    108000.00        216442.62     100.00


In [6]:
!ONEDNN_VERBOSE=1 python run_generation-2.py --device cpu --input-tokens=32 --max-new-tokens 32 \
    --num-iter=15 --num-warmup=5 --batch-size=1 -m EleutherAI/gpt-j-6b \
    --dtype bfloat16 --profile > /tmp/gptj-prof_2.log 2>/dev/null

In [7]:
!tail -n 25 /tmp/gptj-prof_2.log 
!echo ====================
!/home/taosy/repo/github/oneDNN/scripts/verbose_converter/verbose_converter.py -i /tmp/gptj-prof_2.log  -g breakdown -k prim_kind  impl | column -t -s,

onednn_verbose,exec,cpu,matmul,gemm:jit,undef,src_bf16::blocked:ab:f0 wei_bf16::blocked:ab:f0 dst_bf16::blocked:ab:f0,attr-scratchpad:user ,,4x4096:4096x4096:4x4096,0.712891
onednn_verbose,exec,cpu,matmul,gemm:jit,undef,src_bf16::blocked:ab:f0 wei_bf16::blocked:ab:f0 dst_bf16::blocked:ab:f0,attr-scratchpad:user attr-post-ops:sum ,,4x4096:4096x16384:4x16384,2.04321
onednn_verbose,exec,cpu,matmul,gemm:jit,undef,src_bf16::blocked:ab:f0 wei_bf16::blocked:ab:f0 dst_bf16::blocked:ab:f0,attr-scratchpad:user attr-post-ops:sum ,,4x16384:16384x4096:4x4096,3.08203
onednn_verbose,exec,cpu,matmul,gemm:jit,undef,src_bf16::blocked:ab:f0 wei_bf16::blocked:ab:f0 dst_bf16::blocked:ab:f0,attr-scratchpad:user attr-post-ops:sum ,,4x4096:4096x50400:4x50400,5.94385
['Once upon a time, there existed a little girl, who liked to have adventures. She wanted to go to places and meet new people, and have fun. One day, she decided to go on an adventure. She packed her bags, and set off on her journey.\n\nThe little

prim_kind  impl      ncalls  time(ms)   overall%  agg_ncalls(avg)  agg_time(ms)  agg_overall%
matmul     gemm:jit  108000  193801.48  100.00    108000.00        193801.48     100.00


In [9]:
!ONEDNN_VERBOSE=1 python run_generation-2.py --device cpu --input-tokens=32 --max-new-tokens 32 \
    --num-iter=15 --num-warmup=5 --batch-size=1 -m EleutherAI/gpt-j-6b \
    --ipex --dtype bfloat16 --profile > /tmp/gptj-prof_3.log 2>/dev/null

In [10]:
!tail -n 25 /tmp/gptj-prof_3.log 
!echo ====================
!/home/taosy/repo/github/oneDNN/scripts/verbose_converter/verbose_converter.py -i /tmp/gptj-prof_3.log  -g breakdown -k prim_kind  impl | column -t -s,

onednn_verbose,exec,cpu,inner_product,x64:gemm:jit,forward_training,src_bf16::blocked:ab::f0 wei_bf16::blocked:ba::f0 bia_undef::undef::: dst_bf16::blocked:ab::f0,attr-scratchpad:user ,,mb4ic4096oc4096,0.498047
onednn_verbose,exec,cpu,inner_product,x64:gemm:jit,forward_training,src_bf16::blocked:ab::f0 wei_bf16::blocked:ab::f0 bia_bf16::blocked:a::f0 dst_bf16::blocked:ab::f0,attr-scratchpad:user ,,mb4ic4096oc16384,1.06714
onednn_verbose,exec,cpu,inner_product,x64:gemm:jit,forward_training,src_bf16::blocked:ab::f0 wei_bf16::blocked:ba::f0 bia_bf16::blocked:a::f0 dst_bf16::blocked:ab::f0,attr-scratchpad:user ,,mb4ic16384oc4096,2.01001
onednn_verbose,exec,cpu,inner_product,x64:gemm:jit,forward_training,src_bf16::blocked:ab::f0 wei_bf16::blocked:ba::f0 bia_bf16::blocked:a::f0 dst_bf16::blocked:ab::f0,attr-scratchpad:user ,,mb4ic4096oc50400,4.56104
['Once upon a time, there existed a little girl, who liked to have adventures. She wanted to go to places and meet new people, and have fun. One